---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [2]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    # mean value of the target (0 or 1) represents the ratio of spam to not spam
    return spam_data['target'].mean() * 100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer().fit(X_train)

def answer_two():
    
    tokens = pd.DataFrame(cvect.get_feature_names(), columns=['token'])
    tokens['length'] = tokens['token'].apply(len)
    
    return tokens.sort_values('length').iloc[-1][0]

In [6]:
answer_two()

C:\Users\gerry\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    X_train_cvect = cvect.transform(X_train)
    X_test_cvect = cvect.transform(X_test)
    
    multNB_clf = MultinomialNB(alpha=0.1).fit(X_train_cvect, y_train)
    
    y_pred = multNB_clf.predict(X_test_cvect)
    
    return roc_auc_score(y_test, y_pred)

In [8]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    tvect = TfidfVectorizer()
    X_train_tvect = tvect.fit_transform(X_train)
    
    tfidf_df = pd.DataFrame(X_train_tvect.toarray(),columns=tvect.get_feature_names())
    
    tfidf_maxsorted = tfidf_df.agg(max).sort_values()
    
    return tfidf_maxsorted[:20], tfidf_maxsorted[:-21:-1]

In [10]:
answer_four()

C:\Users\gerry\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    
    tvect = TfidfVectorizer(min_df=3)
    X_train_tvect = tvect.fit_transform(X_train)
    X_test_tvect = tvect.transform(X_test)
    
    multNB_clf = MultinomialNB(alpha=0.1).fit(X_train_tvect, y_train)
    y_pred = multNB_clf.predict(X_test_tvect)
    
    return roc_auc_score(y_test,y_pred)

In [12]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    
    spam_data['length'] = spam_data['text'].str.len()
    
    avglen_spam = np.mean(spam_data['length'][spam_data['target'] == 1])
    avglen_notspam = np.mean(spam_data['length'][spam_data['target'] == 0])
    
    return avglen_notspam, avglen_spam

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    
    tvect = TfidfVectorizer(min_df=5)
    
    X_train_tvect = tvect.fit_transform(X_train)
    X_train_tvect_lenfeature = add_feature(X_train_tvect, X_train.str.len())
    
    X_test_tvect = tvect.transform(X_test)
    X_test_tvect_lenfeature = add_feature(X_test_tvect, X_test.str.len())
    
    svc_clf = SVC(C=10000).fit(X_train_tvect_lenfeature, y_train)
    y_pred = svc_clf.predict(X_test_tvect_lenfeature)
    
    return roc_auc_score(y_test, y_pred)

In [17]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    
    spam_data['digit_count'] = spam_data['text'].str.findall('\d').str.len()
    
    avg_dc_spam = spam_data['digit_count'][spam_data['target'] == 1].mean()
    avg_dc_notspam = spam_data['digit_count'][spam_data['target'] == 0].mean()
    
    return avg_dc_notspam, avg_dc_spam

In [19]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    tvect = TfidfVectorizer(min_df=5, ngram_range=(1,3))
    
    X_train_tvect = tvect.fit_transform(X_train)
    X_train_tvect_add = add_feature(X_train_tvect,
                                    [X_train.str.len(), X_train.str.findall('\d').str.len()])
    
    X_test_tvect = tvect.transform(X_test)
    X_test_tvect_add = add_feature(X_test_tvect,
                                   [X_test.str.len(), X_test.str.findall('\d').str.len()])
    
    log_clf = LogisticRegression(C=100).fit(X_train_tvect_add, y_train)
    y_pred = log_clf.predict(X_test_tvect_add)
    
    
    return roc_auc_score(y_test, y_pred)
    #return X_test.str.findall('\d').str.len()
    #return [len(w) for w in X_train]
    #return X_train.str.len().tolist()

In [21]:
answer_nine()

C:\Users\gerry\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9733651087380949

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():
    
    spam_data['non-word_count'] = spam_data['text'].str.findall('\W').str.len()
    
    avg_nwc_spam = spam_data['non-word_count'][spam_data['target'] == 1].mean()
    avg_nwc_notspam = spam_data['non-word_count'][spam_data['target'] == 0].mean()
    
    return avg_nwc_notspam, avg_nwc_spam

In [23]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
def answer_eleven():
    
    cvect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5))

    X_train_cvect = cvect.fit_transform(X_train)

    X_train_cvect_add = add_feature(X_train_cvect, [X_train.str.len(),
                                                          X_train.str.findall('\d').str.len(),
                                                          X_train.str.findall('\W').str.len()])
    
    X_test_cvect = cvect.transform(X_test)
    
    #print(X_test.str.len())
    #print(X_test.str.findall('\d').str.len())
    #print(X_test.str.findall('\W').str.len())
    
    #print(len(X_test_cvect.get_feature_names()))
    
    X_test_cvect_add = add_feature(X_test_cvect, [X_test.str.len(),
                                                          X_test.str.findall('\d').str.len(),
                                                          X_test.str.findall('\W').str.len()])
    
    log_clf = LogisticRegression(C=100).fit(X_train_cvect_add, y_train)
    y_pred = log_clf.predict(X_test_cvect_add)
    
    coef_index = log_clf.coef_[0].argsort()
    features = np.array(cvect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    smallest = features[coef_index[:10]]
    largest = features[coef_index[:-11:-1]]
        
    return roc_auc_score(y_test, y_pred), smallest.tolist(), largest.tolist()

In [25]:
answer_eleven()

C:\Users\gerry\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\gerry\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(0.9813973821367333,
 ['..', '. ', ' i', ' go', ' y', '? ', 'pe', 'go', 'ok', 'h '],
 ['digit_count', 'co', 'ne', 'ww', 'ar', 'ia', ' ch', 'xt', 'mob', 'uk'])